#### 07 - Serving + Demo (Portfolio Style)

This notebook packages the RAG pipeline into a single callable `ask()` function:
1) Embed the question (Azure OpenAI)
2) Retrieve top-k chunks (Option A brute-force or Option B Vector Search)
3) Build a grounded prompt
4) Generate an answer (Azure OpenAI)
5) Log the request/response to `rag_query_logs`
6) (Optional) Evaluate the answer and write to `rag_evaluations`

Output is a JSON-like dict that can later be used for an API or a simple UI.

In [0]:
%run ./00_install_deps_and_restart

In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/948.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━ 747.5/948.6 kB 22.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/361.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [0]:
%run ./00_init_openai_client

In [0]:
import mlflow
# Disable mlflow autologging
mlflow.autolog(disable=True)
mlflow.openai.autolog(disable=True)

In [0]:

# Call to ask:

result = ask("What is difference between a normal Azure VM and ephemeral VM?", k=6, retriever="A", do_eval=True, verbose=False)
print("Query ID:", result["query_id"])
print("\nAnswer:\n", result["answer"])

print("\nTop sources:")
for s in result["sources"][:3]:
    print("-", s["url"])

Query ID: 41c2a3df-d20b-449f-b8e1-e7906dc21b01

Answer:
 The primary difference between a normal Azure VM and an ephemeral VM lies in the storage of the operating system disk and the nature of the disk itself.

1. **Operating System Disk Storage**:
   - **Normal Azure VM**: A standard Azure VM typically uses a managed OS disk that is persistent. This means that the OS disk retains data even after the VM is stopped or deallocated. The OS disk can be backed up and restored, and it is suitable for production workloads where data persistence is critical.
   - **Ephemeral VM**: An ephemeral VM, on the other hand, uses an ephemeral OS disk, which is a temporary disk that is not persistent. The data on this disk is lost when the VM is deallocated or stopped. Ephemeral disks are designed for scenarios where the OS does not need to retain state across reboots, such as stateless applications or development environments.

2. **Performance and Use Cases**:
   - **Normal Azure VM**: These VMs are s

In [0]:
# Try a few:
qs = [
    "What is an availability set used for?",
    "How do I change VM size in Azure?",
    "How do I create a VM from an image?",
]
for q in qs:
    out = ask(q, k=6, retriever="B", do_eval=True, verbose=False)
    print("\n======================")
    print("Q:", q)
    print("A:", out["answer"][:400], "...")
    print("Faithfulness:", out["eval"]["faithfulness"] if out["eval"] else None)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.

Q: What is an availability set used for?
A: An availability set is used to provide high availability to applications deployed on Azure virtual machines (VMs). By placing VMs within an availability set, you ensure that during outages or maintenance events that require a reboot, at least one VM remains available. Availability sets achieve this by spreading individual VMs across multiple update domains (UDs) and fault domains (FDs). 

During s ...
Faithfulness: 4
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please

In [0]:
# Latest logs

spark.sql(f"""
SELECT
  created_at,
  query_id,
  question,
  top_k,
  retrieved_chunks[0].url AS top_source
FROM {RAG_LOG_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()

created_at,query_id,question,top_k,top_source
2026-01-16T07:04:36.879447Z,e52ebe48-7b2e-4395-a708-6ac9fe54d89a,How do I create a VM from an image?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-specialized-image-version.md
2026-01-16T07:04:25.356524Z,4e6421b1-db34-49a4-bd08-b9b91bd7395d,How do I change VM size in Azure?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/disks-migrate-lrs-zrs.md
2026-01-16T07:04:15.259479Z,ecba2ab1-6200-438c-82a3-f9aef64f27ff,What is an availability set used for?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/virtual-machine-scale-sets-orchestration-modes.md
2026-01-16T07:01:49.484794Z,41c2a3df-d20b-449f-b8e1-e7906dc21b01,What is difference between a normal Azure VM and ephemeral VM?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-16T06:49:38.556206Z,c1d014f3-80a0-480d-9e67-75c1c60b9539,What is Azure VM Scale Sets?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/flexible-virtual-machine-scale-sets-powershell.md
2026-01-16T06:49:30.934227Z,534688cd-03c9-4729-8837-e38e27866f33,What is Azure VM Scale Sets?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/flexible-virtual-machine-scale-sets-powershell.md
2026-01-16T06:49:21.01436Z,f6d7f27f-f630-4ea1-a50f-46997963d162,How do I resize an Azure virtual machine?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-customization.md
2026-01-16T06:49:12.399182Z,7f89aade-6341-412c-8289-320fa7ec0a3b,How do I resize an Azure virtual machine?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-customization.md
2026-01-16T06:49:02.860406Z,8d759cd9-f704-4e63-ade5-944c33af3ba5,What is the difference between a normal Azure VM and an ephemeral VM?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-16T06:48:52.963109Z,e9f6e839-c5ad-415b-9cf0-6ad44ecc2a0f,What is the difference between a normal Azure VM and an ephemeral VM?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md


In [0]:
# Latest evaluations

spark.sql(f"""
SELECT
  created_at,
  query_id,
  retrieval_relevance,
  answer_relevance,
  faithfulness,
  notes
FROM {RAG_EVAL_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()

created_at,query_id,retrieval_relevance,answer_relevance,faithfulness,notes
2026-01-16T07:04:39.166459Z,e52ebe48-7b2e-4395-a708-6ac9fe54d89a,5,5,5,"The retrieved excerpts provide detailed and relevant information on creating a VM from a specialized image using both Azure CLI and Azure PowerShell, which directly addresses the question. The answer is well-supported by the excerpts, with no hallucination present."
2026-01-16T07:04:27.808531Z,4e6421b1-db34-49a4-bd08-b9b91bd7395d,2,5,2,"The answer provides a detailed and accurate method for changing VM size in Azure, but the retrieved excerpts do not directly support the answer. The excerpts focus on disk management and other unrelated topics, leading to a lower score for retrieval relevance and faithfulness."
2026-01-16T07:04:18.126449Z,ecba2ab1-6200-438c-82a3-f9aef64f27ff,3,5,4,"The answer is relevant and directly addresses the question about availability sets. However, the retrieved excerpts do not provide direct information about availability sets, which affects the retrieval relevance score. The answer is mostly supported by the context of the excerpts, but there is a slight risk of hallucination since the specific details about availability sets are not explicitly mentioned in the retrieved sources."
2026-01-16T07:01:52.947065Z,41c2a3df-d20b-449f-b8e1-e7906dc21b01,2,5,2,"The answer provides a clear and accurate distinction between normal Azure VMs and ephemeral VMs, addressing the question well. However, the retrieved excerpts do not directly support the answer, as they focus on temporary disks and other VM types without specifically discussing ephemeral VMs."
2026-01-16T06:55:33.693136Z,e9f6e839-c5ad-415b-9cf0-6ad44ecc2a0f,2,5,3,"The answer provides a clear and accurate distinction between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information about the differences between these VM types, which affects the faithfulness score."
2026-01-16T06:55:31.106366Z,8d759cd9-f704-4e63-ade5-944c33af3ba5,2,5,3,"The answer provides a clear and accurate distinction between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information about the differences between these VM types, which affects the faithfulness score as the answer is not directly supported by the provided excerpts."
2026-01-16T06:55:28.378318Z,7f89aade-6341-412c-8289-320fa7ec0a3b,3,5,4,"The retrieved excerpts provide some relevant information about resizing VMs, including the need for a reboot and that certain marketplace offerings are not supported. However, the answer includes more detailed steps and context that are not fully captured in the excerpts, leading to a slightly lower score for faithfulness."
2026-01-16T06:55:25.811058Z,f6d7f27f-f630-4ea1-a50f-46997963d162,3,5,4,"The answer provides a comprehensive guide on how to resize an Azure virtual machine, which is relevant to the question. However, the retrieved excerpts do not directly support the detailed steps provided in the answer, particularly regarding the resizing process. The first excerpt mentions the need for downtime and rebooting, which aligns with the answer, but it lacks specific instructions on how to resize using the Azure Portal, CLI, or PowerShell."
2026-01-16T06:55:21.246305Z,534688cd-03c9-4729-8837-e38e27866f33,5,5,5,"The retrieved excerpts provide relevant information about Azure VM Scale Sets, including their features, orchestration modes, and scaling capabilities, which directly support the answer provided."
2026-01-16T06:55:18.099844Z,c1d014f3-80a0-480d-9e67-75c1c60b9539,5,5,5,"The retrieved excerpts provide relevant information about Azure VM Scale Sets, including their features and management, which directly supports the answer given."
